### Downloading dataset

In [2]:
#!wget -nc -O ../Datasets/3_crimes.csv https://huggingface.co/datasets/gymprathap/Chicago-Crime-Dataset/resolve/main/Crimes_-_2001_to_Present_20240713.csv


### Spark setup

In [3]:
# Basic imports
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [4]:
# Build SparkSession
spark = SparkSession.builder \
    .appName("CrimesFix") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.sql.shuffle.partitions", "16") \
    .getOrCreate()

## Inicial data inspection

### Importing dataset

In [5]:
# Reading data
file_crimes = '/home/jovyan/code/Projeto/Datasets/3_crimes.csv'

! head $file_crimes
df_crimes = spark.read.csv(file_crimes, header=True, sep=',', inferSchema=True)


ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
11037294,JA371270,03/18/2015 12:00:00 PM,0000X W WACKER DR,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,BANK,false,false,0111,001,42,32,11,,,2015,08/01/2017 03:52:26 PM,,,
11646293,JC213749,12/20/2018 03:00:00 PM,023XX N LOCKWOOD AVE,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,APARTMENT,false,false,2515,025,36,19,11,,,2018,04/06/2019 04:04:43 PM,,,
11645836,JC212333,05/01/2016 12:25:00 AM,055XX S ROCKWELL ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,,false,false,0824,008,15,63,11,,,2016,04/06/2019 04:04:43 PM,,,
11645959,JC211511,12/20/2018 04:00:00 PM,045XX N ALBANY AVE,2820,OTHER OFFENSE,TELEPHONE THREAT,RESIDENCE,false,false,1724,017,33,14,08A,,,2018,04/06/2019 04:04:43 PM,,,
11645601,JC212935,06/01/2014 12:01:00 AM,087XX S SANG

### Check top 10 rows of the inicial dataset and atributes' datatypes

In [6]:
df_crimes.show(10)
print(f'df_crimes - number of rows is {df_crimes.count()    }.')
df_crimes.printSchema()

+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|                Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11037294|   JA371270|03/18/2015 12:00:...|   0000X W WACKER DR|1153|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|                BANK| false|   false| 111|       1|  42|            32|      11| 

### Duplicates and NULLs

In [7]:
# Now let's check the number of duplicated rows
print(f'df_crimes - number of rows is {df_crimes.count()  }; after dropDuplicates() applied would be {df_crimes.dropDuplicates().count()   }.')

df_crimes - number of rows is 2364242; after dropDuplicates() applied would be 2364242.


In [8]:
# Check for null values
print(f'''df_crimes - number of rows after dropna(how='any') would be {df_crimes.dropna(how='any').count()     }.''')

df_crimes - number of rows after dropna(how='any') would be 2270795.


In [9]:
# There is half a million of nulls in the dataset, so let's check how many nulls are in each column
print('Checking nulls at each column of df_crimes')
dict_nulls_retail = {col: df_crimes.filter(df_crimes[col].isNull()).count() for col in df_crimes.columns}
dict_nulls_retail

Checking nulls at each column of df_crimes


{'ID': 0,
 'Case Number': 0,
 'Date': 0,
 'Block': 0,
 'IUCR': 0,
 'Primary Type': 0,
 'Description': 0,
 'Location Description': 6243,
 'Arrest': 0,
 'Domestic': 0,
 'Beat': 0,
 'District': 37,
 'Ward': 57174,
 'Community Area': 57308,
 'FBI Code': 0,
 'X Coordinate': 32551,
 'Y Coordinate': 32551,
 'Year': 0,
 'Updated On': 0,
 'Latitude': 32552,
 'Longitude': 32552,
 'Location': 32552}

In [10]:
#We see that the last 5 columns about location have a lot of nulls, so let's drop them, as they are not useful for our analysis. Because otherwise nulls removing can affect filled rows.

# Drop the specified columns from df_crimes
columns_to_drop = ['X Coordinate', 'Y Coordinate', 'Latitude', 'Longitude', 'Location']
df_crimes = df_crimes.drop(*columns_to_drop)

# Verify the remaining columns
df_crimes.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)



In [11]:
# Check for null values again
print(f'''df_crimes - number of rows before {df_crimes.count()} after dropna(how='any') would be {df_crimes.dropna(how='any').count()     }.''')

df_crimes - number of rows before 2364242 after dropna(how='any') would be 2300379.


### Drop null values and make sure everything is clear

In [12]:
df_crimes = df_crimes.dropna(how='any')

In [ ]:
print('Checking nulls at each column of df_crimes')
dict_nulls_retail = {col: df_crimes.filter(df_crimes[col].isNull()).count() for col in df_crimes.columns}
dict_nulls_retail

Checking nulls at each column of df_crimes


### Unique values

In [ ]:
print('\nUniqueness of values:')
number_records = df_crimes.count()
cols_interest = df_crimes.columns
for cl in cols_interest:
    k = df_crimes.select(cl).distinct().count()
    print(f'Column {cl} has {k} unique values out of {number_records} records.')


Uniqueness of values:
Column ID has 7475294 unique values out of 7475294 records.


Column Case Number has 7474737 unique values out of 7475294 records.
Column Date has 3103277 unique values out of 7475294 records.
Column Block has 39071 unique values out of 7475294 records.
Column IUCR has 404 unique values out of 7475294 records.
Column Primary Type has 35 unique values out of 7475294 records.
Column Description has 551 unique values out of 7475294 records.
Column Location Description has 217 unique values out of 7475294 records.
Column Arrest has 2 unique values out of 7475294 records.
Column Domestic has 2 unique values out of 7475294 records.
Column Beat has 304 unique values out of 7475294 records.
Column District has 24 unique values out of 7475294 records.
Column Ward has 50 unique values out of 7475294 records.
Column Community Area has 78 unique values out of 7475294 records.
Column FBI Code has 27 unique values out of 7475294 records.
Column Year has 24 unique values out of 7475294 records.
Column Updated On has 6364 unique values out of 7475294 records.


## Attributes review after a data inspection + necessary data transformation

#### Sometimes values may contain spaces, especially when loading from CSV. This may affect the number of unique values ("051A" ≠ "051A "). We check if there are such cases.

In [ ]:
from pyspark.sql.types import StringType

# Получаем все строковые колонки
string_columns = [field.name for field in df_crimes.schema.fields if isinstance(field.dataType, StringType)]

# Проверим наличие лишних пробелов
for col_name in string_columns:
    count_trim_diff = df_crimes.filter(F.col(col_name) != F.trim(F.col(col_name))).count()
    if count_trim_diff > 0:
        print(f"Column '{col_name}' has {count_trim_diff} value(s) with leading or trailing spaces.")
    else:
        print(f"Column '{col_name}' has no extra spaces.")

Column 'Case Number' has no extra spaces.
Column 'Date' has no extra spaces.
Column 'Block' has no extra spaces.
Column 'IUCR' has no extra spaces.
Column 'Primary Type' has no extra spaces.
Column 'Description' has no extra spaces.
Column 'Location Description' has no extra spaces.
Column 'FBI Code' has no extra spaces.
Column 'Updated On' has no extra spaces.


### After a primary review of the variables and cleaning dataset from NULLs, we can execute detailed data exploration and transformation. 
### Let's break it down by attributes:

#### ID

This attribute represents the number of a row. Considering our work as a ML task, we do not need this column, therefore it can be removed.

In [ ]:
df_crimes = df_crimes.drop("ID")
df_crimes.printSchema()

root
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)



#### Case Number

This attribute represents the official case number assigned by the police. As we can see, amount of case numbers is lower than the amount of rows, this can happen if:
- the same case involves more than one offense;
- there are multiple locations or suspects;
- there are different actions recorded for the same incident... 

Although number of them is small, we have to remove all of this cases because we can not review each of them and conclude the reason of duplicated case number.

**Therefore, we remove all cases where multiple rows belong to the same Case Number.**
**After that we remove this column, because it is irrelevant for the model.**

In [ ]:
# First find Case Numbers that occur exactly once
case_counts = (
    df_crimes
    .groupBy("Case Number")
    .agg(F.count("*").alias("cnt"))
    .filter("cnt = 1")
    .select("Case Number")
)

# Now leave only those strings that refer to unique Case Number
df_crimes = df_crimes.join(case_counts, on="Case Number", how="inner")

k = df_crimes.select('Case Number').distinct().count()
print(f'Column Case Number has {k} unique values out of {number_records} records.')

Column Case Number has 7474272 unique values out of 7475294 records.


In [ ]:
df_crimes = df_crimes.drop('Case Number')

#### Date

This attribute reflects the date and time of the offense in timestamp format. \
This information is very important. However, in order to analyze it, it is necessary to manipulate the data into an acceptable form. \
Let's break the data into pieces of Month, Day, Hour, Minute. Since we have Year column already in the initial dataset, we will move it upfront, so date columns will be placed together. \
After extracting parts of the Date, we can remove this column.

In [ ]:
# Convert the Date column to timestamp and extract components
df_crimes = df_crimes.withColumn('Timestamp', F.to_timestamp(df_crimes['Date'], 'MM/dd/yyyy hh:mm:ss a'))
df_crimes = df_crimes.withColumn('Hour', F.hour(df_crimes['Timestamp']))
df_crimes = df_crimes.withColumn('Minute', F.minute(df_crimes['Timestamp']))
df_crimes = df_crimes.withColumn('Day', F.dayofmonth(df_crimes['Timestamp']))
df_crimes = df_crimes.withColumn('Month', F.month(df_crimes['Timestamp']))
df_crimes = df_crimes.drop('Timestamp')

#Make correct column order and removing date
desired_order = [
    "Year", "Month", "Day", "Hour", "Minute"
]
remaining_columns = [col for col in df_crimes.columns if col not in desired_order + ["Date"]]
new_column_order = desired_order + remaining_columns
df_crimes = df_crimes.select(*new_column_order)

#### Block

The street where the crime occurred, but not the full address - most often rounded up to the nearest block (for example: 010XX W 59TH ST). \ 
It was concidered unnecessary to extract something from this attribute and use it, due to its preciseness. For the geographical contextualization, we have other column to consider.

In [ ]:
df_crimes = df_crimes.drop('Block')

#### IUCR

The code assigned to the type of offense under the Illinois UCR system.

**Check whether the IUCR variable contains only digits.** This will help us to understand whether we can consider it as a numeric variable or leave it as a string. When previewing the values, we noticed the presence of letter suffixes (e.g., 051A, 142B), which makes the variable categorical.

In [ ]:
df_crimes.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Day: integer (nullable = true)
 |-- Hour: integer (nullable = true)
 |-- Minute: integer (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- Updated On: string (nullable = true)



In [ ]:
# Проверка значений, не состоящих только из цифр
non_digit_rows = df_crimes.filter(~F.col("IUCR").rlike("^[0-9]+$"))
non_digit_rows.select("IUCR").distinct().show(5, truncate=False)

# Подсчёт количества таких строк
print("Non-digit IUCR values:", non_digit_rows.count())

+----+
|IUCR|
+----+
|031A|
|051B|
|031B|
|502R|
|033A|
+----+
only showing top 5 rows

Non-digit IUCR values: 358014


IUCR is a code from the Illinois Uniform Crime Reporting system that may contain letters and is not a numeric characteristic. It cannot be used as a number because 051A, 051B, and 051C are different types of offenses.\
**Leave IUCR as a string and prepare it for indexing.**

For use in machine learning models, categorical values need to be converted to numeric values. We use StringIndexer, which creates a new column with indexes.

In [ ]:
from pyspark.ml.feature import StringIndexer

# IUCR indexing (categorical → numeric)
indexer = StringIndexer(inputCol="IUCR", outputCol="IUCR Num")
df_crimes = indexer.fit(df_crimes).transform(df_crimes)
df_crimes = df_crimes.withColumn("IUCR Num", F.col("IUCR Num").cast("int"))

In [ ]:
#Change columns order
desired_order = [
    "Year", "Month", "Day", "Hour", "Minute", "IUCR Num", 'IUCR'
]
remaining_columns = [col for col in df_crimes.columns if col not in desired_order]
new_column_order = desired_order + remaining_columns
df_crimes = df_crimes.select(*new_column_order)

#### Primary Type

The primary type of crime. We have 35 types in our dataset. \
Have to check the unique values of a column, for contextual understanding of the dataset. \
Then create a numeric interpretation of this categorical attribute.

In [ ]:
# Display all types of crimes (35)
df_crimes.select("Primary Type").distinct().show(35, truncate=False)

+---------------------------------+
|Primary Type                     |
+---------------------------------+
|OFFENSE INVOLVING CHILDREN       |
|HUMAN TRAFFICKING                |
|CRIMINAL DAMAGE                  |
|PUBLIC PEACE VIOLATION           |
|NON-CRIMINAL (SUBJECT SPECIFIED) |
|ROBBERY                          |
|KIDNAPPING                       |
|STALKING                         |
|INTIMIDATION                     |
|CONCEALED CARRY LICENSE VIOLATION|
|HOMICIDE                         |
|MOTOR VEHICLE THEFT              |
|CRIM SEXUAL ASSAULT              |
|LIQUOR LAW VIOLATION             |
|THEFT                            |
|ASSAULT                          |
|PROSTITUTION                     |
|WEAPONS VIOLATION                |
|INTERFERENCE WITH PUBLIC OFFICER |
|GAMBLING                         |
|NON - CRIMINAL                   |
|BATTERY                          |
|OBSCENITY                        |
|PUBLIC INDECENCY                 |
|DECEPTIVE PRACTICE         

In [ ]:
# Create an indexer
indexer = StringIndexer(inputCol="Primary Type", outputCol="Primary Type Num")
# Train the indexer and transform the data
df_crimes = indexer.fit(df_crimes).transform(df_crimes)
df_crimes = df_crimes.withColumn("Primary Type Num", F.col("Primary Type Num").cast("int"))

In [ ]:
#Make correct column order
desired_order = [
    'Year', 'Month', 'Day', 'Hour', 'Minute', 'IUCR Num', 'IUCR', 'Primary Type Num', 'Primary Type'
]
remaining_columns = [col for col in df_crimes.columns if col not in desired_order]
new_column_order = desired_order + remaining_columns
df_crimes = df_crimes.select(*new_column_order)
df_crimes.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Day: integer (nullable = true)
 |-- Hour: integer (nullable = true)
 |-- Minute: integer (nullable = true)
 |-- IUCR Num: integer (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type Num: integer (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- Updated On: string (nullable = true)



#### Description

To understand the data contained in this column, let's first of all check some of unique values

In [ ]:
df_crimes.select('Description').distinct().show(truncate=False)

+------------------------------+
|Description                   |
+------------------------------+
|DOMESTIC BATTERY SIMPLE       |
|UNLAWFUL POSS OF HANDGUN      |
|ATT CRIM SEXUAL ABUSE         |
|TO RESIDENCE                  |
|ATTEMPT ARSON                 |
|CRIMINAL TRANSMISSION OF HIV  |
|PRO EMP HANDS NO/MIN INJURY   |
|UNLAWFUL USE/SALE AIR RIFLE   |
|MANU/DELIVER:AMPHETAMINES     |
|BOMB THREAT                   |
|ENDANGERING LIFE/HEALTH CHILD |
|ATTEMPT: ARMED-OTHER DANG WEAP|
|BY EXPLOSIVE                  |
|ATT: TRUCK, BUS, MOTOR HOME   |
|ATT AGG CRIMINAL SEXUAL ABUSE |
|FALSE FIRE ALARM              |
|OBSTRUCTING SERVICE           |
|AGG PRO.EMP: OTHER DANG WEAPON|
|AGGRAVATED OF A CHILD         |
|UNLAWFUL USE OF A COMPUTER    |
+------------------------------+
only showing top 20 rows



It was decided to exclude the Description column. \
This attribute contains about 551 unique values and represents a refinement to the more generalized attribute Primary Type. \
For this work, it was decided to focus on generalized crime categories. \

In [ ]:
df_crimes = df_crimes.drop("Description")

#### Location Description

Where exactly did the crime take place. To clearly understand the attribute, let's check some of unique values

In [ ]:
df_crimes.select('Location Description').distinct().show(truncate=False)

+-----------------------------------------+
|Location Description                     |
+-----------------------------------------+
|CHA HALLWAY                              |
|GAS STATION                              |
|CURRENCY EXCHANGE                        |
|CTA PLATFORM                             |
|SPORTS ARENA/STADIUM                     |
|CHURCH / SYNAGOGUE / PLACE OF WORSHIP    |
|TAVERN                                   |
|AIRPORT PARKING LOT                      |
|FARM                                     |
|VEHICLE - COMMERCIAL: TROLLEY BUS        |
|RESIDENCE                                |
|HOUSE                                    |
|VEHICLE NON-COMMERCIAL                   |
|APPLIANCE STORE                          |
|HOSPITAL                                 |
|SCHOOL - PRIVATE GROUNDS                 |
|OTHER RAILROAD PROPERTY / TRAIN DEPOT    |
|CTA PARKING LOT / GARAGE / OTHER PROPERTY|
|COLLEGE / UNIVERSITY - RESIDENCE HALL    |
|PARKING LOT/GARAGE(NON.RESID.) 

In [ ]:
indexer = StringIndexer(
    inputCol="Location Description", 
    outputCol="Location Description Num"
)
df_crimes = indexer.fit(df_crimes).transform(df_crimes)
df_crimes = df_crimes.withColumn("Location Description Num", F.col("Location Description Num").cast("int"))
df_crimes.select("Location Description", "Location Description Num").show(10, truncate=False)

+--------------------+------------------------+
|Location Description|Location Description Num|
+--------------------+------------------------+
|SIDEWALK            |3                       |
|APARTMENT           |2                       |
|APARTMENT           |2                       |
|CHA STAIRWELL       |172                     |
|STREET              |0                       |
|OTHER               |4                       |
|STREET              |0                       |
|STREET              |0                       |
|STREET              |0                       |
|RESIDENCE           |1                       |
+--------------------+------------------------+
only showing top 10 rows



In [ ]:
#Make correct column order
desired_order = [
    'Year', 'Month', 'Day', 'Hour', 'Minute', 'IUCR Num', 'IUCR', 'Primary Type Num', 'Primary Type', 'Location Description Num', 'Location Description'
]
remaining_columns = [col for col in df_crimes.columns if col not in desired_order]
new_column_order = desired_order + remaining_columns
df_crimes = df_crimes.select(*new_column_order)
df_crimes.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Day: integer (nullable = true)
 |-- Hour: integer (nullable = true)
 |-- Minute: integer (nullable = true)
 |-- IUCR Num: integer (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type Num: integer (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Location Description Num: integer (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- Updated On: string (nullable = true)



#### Arrest

Boolean column (true/false), indicates whether an arrest was made in this case. \
We need to transform it into a binary column (1/0)

In [ ]:
# Convert the Boolean column to a binary numeric column (0/1)
df_crimes = df_crimes.withColumn("Arrest", F.col("Arrest").cast("int"))

#### Domestic

Boolean column (true/false), indicates whether the crime is related to domestic violence. \
We need to transform it into a binary column (1/0)

In [ ]:
# Convert the Boolean column to a binary numeric column (0/1)
df_crimes = df_crimes.withColumn("Domestic", F.col("Domestic").cast("int"))

#### Beat

The smallest unit of a geographic division of the police force.\
Each Beat is part of a specific police district.\
Useful for operational analytics.\
It was decided not to make any manipulations and not to delete the column, as it may play an important role in the model, due to the fact that the operation of different units leads to different outcomes and there may be a pattern in it.

#### District, Ward, Community Area

These two geographical features indicate the following context:\
- **District:** Police District. A wider area that includes several Beats.
- **Ward:** A political-administrative district of Chicago. Used for municipal government.
- **Community Area:** A geographic unit of a city used for city planning.

All of the columns remain without any changes.

#### FBI Code

The FBI's categorization of the offense. Provides a standard for comparison with other regions and for aggregating data at the national level.\
Column is indexed beacause the codes are strings.

In [ ]:
# FBI_Code indexing (categorical → numeric)
indexer = StringIndexer(inputCol="FBI Code", outputCol="FBI Code Num")
df_crimes = indexer.fit(df_crimes).transform(df_crimes)
df_crimes = df_crimes.withColumn("FBI Code Num", F.col("FBI Code Num").cast("int"))

#### Updated On

The date on which the record was updated in the database.\
Column is irrelevant for this work therefore is removed.

In [ ]:
df_crimes = df_crimes.drop('Updated On')

#### Transform column names

In [ ]:
# In order not to have errors with the column names, let's rename them by replacing spaces with underscores
df_crimes = df_crimes.select([F.col(column).alias(column.replace(" ", "_")) for column in df_crimes.columns])
df_crimes.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Day: integer (nullable = true)
 |-- Hour: integer (nullable = true)
 |-- Minute: integer (nullable = true)
 |-- IUCR_Num: integer (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary_Type_Num: integer (nullable = true)
 |-- Primary_Type: string (nullable = true)
 |-- Location_Description_Num: integer (nullable = true)
 |-- Location_Description: string (nullable = true)
 |-- Arrest: integer (nullable = true)
 |-- Domestic: integer (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community_Area: integer (nullable = true)
 |-- FBI_Code: string (nullable = true)
 |-- FBI_Code_Num: integer (nullable = true)



### Quick data check after transformations

#### Unique values and columns list

In [ ]:
print('\nUniqueness of values:')
number_records = df_crimes.count()
cols_interest = df_crimes.columns
for cl in cols_interest:
    k = df_crimes.select(cl).distinct().count()
    print(f'Column {cl} has {k} unique values out of {number_records} records.')


Uniqueness of values:
Column Year has 24 unique values out of 7474272 records.
Column Month has 12 unique values out of 7474272 records.
Column Day has 31 unique values out of 7474272 records.
Column Hour has 24 unique values out of 7474272 records.
Column Minute has 60 unique values out of 7474272 records.
Column IUCR_Num has 404 unique values out of 7474272 records.
Column IUCR has 404 unique values out of 7474272 records.
Column Primary_Type_Num has 35 unique values out of 7474272 records.
Column Primary_Type has 35 unique values out of 7474272 records.
Column Location_Description_Num has 216 unique values out of 7474272 records.
Column Location_Description has 216 unique values out of 7474272 records.
Column Arrest has 2 unique values out of 7474272 records.
Column Domestic has 2 unique values out of 7474272 records.
Column Beat has 304 unique values out of 7474272 records.
Column District has 24 unique values out of 7474272 records.
Column Ward has 50 unique values out of 7474272

In [ ]:
#Check data types
df_crimes.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Day: integer (nullable = true)
 |-- Hour: integer (nullable = true)
 |-- Minute: integer (nullable = true)
 |-- IUCR_Num: integer (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary_Type_Num: integer (nullable = true)
 |-- Primary_Type: string (nullable = true)
 |-- Location_Description_Num: integer (nullable = true)
 |-- Location_Description: string (nullable = true)
 |-- Arrest: integer (nullable = true)
 |-- Domestic: integer (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community_Area: integer (nullable = true)
 |-- FBI_Code: string (nullable = true)
 |-- FBI_Code_Num: integer (nullable = true)



## Data is ready!

## Statistical data exploration

#### Create dataframes containing numerical and categorical attributes

In [ ]:
from pyspark.sql.types import NumericType, StringType, BooleanType

# Get the schema of the dataframe
schema = df_crimes.schema

# Identify numerical and categorical columns
# (assuming numerical columns are of type NumericType and categorical columns are of type StringType or BooleanType)
numerical_columns = [field.name for field in schema if isinstance(field.dataType, NumericType)]
categorical_columns = [field.name for field in schema if isinstance(field.dataType, (StringType, BooleanType))]

# Form two dataframes
numerical_df = df_crimes.select(*numerical_columns)
categorical_df = df_crimes.select(*categorical_columns)

print("Numerical columns:", numerical_columns)
print("Categorical columns:", categorical_columns)

Numerical columns: ['Year', 'Month', 'Day', 'Hour', 'Minute', 'IUCR_Num', 'Primary_Type_Num', 'Location_Description_Num', 'Arrest', 'Domestic', 'Beat', 'District', 'Ward', 'Community_Area', 'FBI_Code_Num']
Categorical columns: ['IUCR', 'Primary_Type', 'Location_Description', 'FBI_Code']


#### Statistics of numerical variables

In [ ]:
desc_df = numerical_df.describe()

# Get all columns except 'summary'
stat_cols = [c for c in desc_df.columns if c != 'summary']

# For each column, round values in 'mean' and 'stddev' rows
for c in stat_cols:
    desc_df = desc_df.withColumn(
        c,
        F.when(F.col("summary").isin("mean", "stddev"), F.round(F.col(c).cast("double"), 2)).otherwise(F.col(c))
    )

desc_df.show()

+-------+-------+-------+-------+-------+-------+--------+----------------+------------------------+-------+--------+-------+--------+-------+--------------+------------+
|summary|   Year|  Month|    Day|   Hour| Minute|IUCR_Num|Primary_Type_Num|Location_Description_Num| Arrest|Domestic|   Beat|District|   Ward|Community_Area|FBI_Code_Num|
+-------+-------+-------+-------+-------+-------+--------+----------------+------------------------+-------+--------+-------+--------+-------+--------------+------------+
|  count|7474272|7474272|7474272|7474272|7474272| 7474272|         7474272|                 7474272|7474272| 7474272|7474272| 7474272|7474272|       7474272|     7474272|
|   mean|2011.22|    6.6|  15.64|  13.07|  19.64|   21.38|            3.73|                    7.71|   0.25|    0.17|1181.39|   11.29|  22.77|         37.43|        3.95|
| stddev|   6.36|   3.33|   8.85|   6.76|  18.53|    34.3|            3.99|                   14.41|   0.43|    0.38| 702.74|    6.95|  13.85|   

#### Statistics of categorical variables

In [ ]:
cat_desc_df = categorical_df.describe()

# Get all columns except 'summary'
stat_cols = [c for c in cat_desc_df.columns if c != 'summary']

# For each column, round values in 'mean' and 'stddev' rows
for c in stat_cols:
    cat_desc_df = cat_desc_df.withColumn(
        c,
        F.when(F.col("summary").isin("mean", "stddev"), F.round(F.col(c).cast("double"), 2)).otherwise(F.col(c))
    )

cat_desc_df.show()

+-------+-------+-----------------+--------------------+--------+
|summary|   IUCR|     Primary_Type|Location_Description|FBI_Code|
+-------+-------+-----------------+--------------------+--------+
|  count|7474272|          7474272|             7474272| 7474272|
|   mean|1115.55|             NULL|                NULL|    12.0|
| stddev| 806.26|             NULL|                NULL|    7.28|
|    min|   0110|            ARSON|"CTA ""L"" PLATFORM"|     01A|
|    max|   5132|WEAPONS VIOLATION|                YMCA|      27|
+-------+-------+-----------------+--------------------+--------+



## Charts

#### Correlations chart

In [ ]:
%pip install plotly

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation
import pandas as pd
import plotly.express as px

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Correlations

# Checking correlation between numeric columns (no NULLs)

col_corr = numerical_df
# Correlation needs vector columns, so we need to convert the data frame to a vector
vector_col = 'corr_features'
assembler = VectorAssembler(inputCols=col_corr.columns, outputCol=vector_col)
df_vector = assembler.transform(df_crimes).select(vector_col)
# Get the correlation matrix - it can be Pearson's (default) or Spearman's
corr = Correlation.corr(df_vector, vector_col).head()
corr_matrix = Correlation.corr(df_vector, vector_col).collect()[0][0].toArray().tolist()

# Convert correlation matrix to a DataFrame for visualization
corr_df = pd.DataFrame(corr_matrix, index=col_corr.columns, columns=col_corr.columns)

# Plot the heatmap using plotly.express
fig = px.imshow(
    corr_df,
    text_auto=True,
    color_continuous_scale=[
        [0.0, "white"], 
        [0.5, "#fca9a9"], 
        [1.0, "darkred"]
    ],
    zmin=-1, zmax=1,
    title="Correlation Matrix",
    width=1000, # graph width
    height=1000, # height of the chart
)

# Optional: reduce the gaps between cells
fig.update_traces(xgap=2, ygap=2)

# Increase the readability of the axis text
fig.update_layout(
    xaxis=dict(tickangle=45), 
    font=dict(size=12),
)

fig.show()

#### Crimes dynamics by Year

In [ ]:
# Grouping by year
agg_year = df_crimes.groupBy('Year').count().orderBy('Year')
agg_year_pd = agg_year.toPandas()

fig = px.line(
    agg_year_pd,
    x='Year',
    y='count',
    markers=True,
    title='Number of Crimes per Year in Chicago'
)
fig.update_layout(xaxis=dict(dtick=1))  # Display each year by X
fig.show()

#### Number of Crimes by Type

In [ ]:
agg_primary_type = df_crimes.groupBy('Primary_Type').count().orderBy(F.desc('count'))
agg_primary_type_pd = agg_primary_type.limit(10).toPandas()
fig2 = px.bar(agg_primary_type_pd, x='Primary_Type', y='count', title='Crimes by Type')
fig2.update_xaxes(tickangle=0)
fig2.show()

#### Arrest vs Non-arrest pie-chart

In [ ]:
agg_arrest = df_crimes.groupBy('Arrest').count()
agg_arrest_pd = agg_arrest.toPandas()
fig3 = px.pie(agg_arrest_pd, names='Arrest', values='count', title='Arrests vs No Arrests')
fig3.show()

#### Arrest by Crime Type

In [ ]:
# Получаем топ-10 по типу преступлений
top_10_types_df = df_crimes.groupBy('Primary_Type') \
    .count() \
    .orderBy(F.desc('count')) \
    .limit(10)

top_10_types = [row['Primary_Type'] for row in top_10_types_df.collect()]

# Группировка по типу и аресту
agg_arrest_by_type = df_crimes.filter(F.col('Primary_Type').isin(top_10_types)) \
    .groupBy('Primary_Type', 'Arrest') \
    .count()

# Преобразуем в pandas
agg_arrest_by_type_pd = agg_arrest_by_type.toPandas()

# Сортировка категорий вручную
total_counts = agg_arrest_by_type_pd.groupby("Primary_Type")["count"].sum().sort_values(ascending=False)
sorted_types = total_counts.index.tolist()
agg_arrest_by_type_pd["Primary_Type"] = pd.Categorical(
    agg_arrest_by_type_pd["Primary_Type"],
    categories=sorted_types,
    ordered=True
)

# Преобразуем Arrest в строку для явного управления цветами
agg_arrest_by_type_pd["Arrest"] = agg_arrest_by_type_pd["Arrest"].map({0: "Not Arrested", 1: "Arrested"})

# Построение графика
fig = px.bar(
    agg_arrest_by_type_pd,
    x='Primary_Type',
    y='count',
    color='Arrest',
    title='Top 10 Crime Types and Arrests',
    barmode='stack',
    category_orders={
        "Primary_Type": sorted_types,
        "Arrest": ["Not Arrested", "Arrested"]  # Arrested — наверху
    },
    color_discrete_map={
        "Arrested": "yellow",
        "Not Arrested": "darkblue"
    }
)
fig.update_xaxes(tickangle=0)
fig.show()

#### Crimes by hour of day

In [ ]:
agg_by_hour = df_crimes.groupBy('Hour').count().orderBy('Hour')
agg_by_hour_pd = agg_by_hour.toPandas()
fig = px.bar(agg_by_hour_pd, x='Hour', y='count', title='Number of Crimes by Hour')

fig.update_layout(
    xaxis=dict(
        dtick=1,
        tickfont=dict(size=10)
    )
)

fig.show()

## Saving data

In [ ]:
# Create a smaller sapling of data and saving it too

seed_to_use = 5
with_replacement = False
sampling_fraction = 0.3
df_crimes_small = df_crimes.sample(
                        withReplacement=with_replacement,
                        fraction=sampling_fraction,
                        seed=seed_to_use
)

df_crimes.write.mode('overwrite').parquet('../Datasets/3_crimes_cleaned')
df_crimes_small.write.mode('overwrite').parquet('../Datasets/3_crimes_cleaned_small')